https://stepik.org/lesson/303592/step/5

In [ ]:
import torch
import torch.nn as nn

eps = 1e-3

input_channels = 3
batch_size = 3
height = 10
width = 10

batch_norm_2d = nn.BatchNorm2d(input_channels, affine=False, eps=eps)

input_tensor = torch.randn(batch_size, input_channels, height, width, dtype=torch.float)


def custom_batch_norm2d(input_tensor, eps):
    # Напишите в этом методе нормирование тензора.

    # Способ 1. Не работает в версии платформы
    # mean = input_tensor.mean(dim=(0, 2, 3), keepdim=True)
    # var = input_tensor.var(dim=(0, 2, 3), keepdim=True, unbiased=False)
    # normed_tensor = (input_tensor - mean) / torch.sqrt(var + eps)

    # Способ 2.Через .cat и .unsqueeze
    # mean = torch.cat([torch.mean(input_tensor[:,i,:,:]).unsqueeze(0) for i in range(input_tensor.shape[1])], dim=0) # цикл по каналам
    # var = torch.cat([torch.var(input_tensor[:,i,:,:], unbiased=False).unsqueeze(0) for i in range(input_tensor.shape[1])], dim=0)
    # normed_tensor = torch.cat([((input_tensor[:,i,:,:] - mean[i])/torch.sqrt(var[i]+eps)).unsqueeze(1) for i in range(input_tensor.shape[1])], dim=1)

    # Способ 3. Через .stack
    mean = torch.tensor([torch.mean(input_tensor[:,i,:,:]) for i in range(input_tensor.shape[1])])
    var = torch.tensor([torch.var(input_tensor[:,i,:,:], unbiased=False) for i in range(input_tensor.shape[1])])
    normed_tensor = torch.stack([(input_tensor[:, i, :, :] - mean[i])
    / torch.sqrt(var[i] + eps)for i in range(input_tensor.shape[1])], dim=1)

    return normed_tensor


# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
# norm_output = batch_norm_2d(input_tensor)
# custom_output = custom_batch_norm2d(input_tensor, eps)
# print(torch.allclose(norm_output, custom_output) and norm_output.shape == custom_output.shape)

True


# Объяснение для Способа 2
`unsqueeze(0)` не участвует непосредственно в вычислении среднего значения. Он используется для **подготовки тензоров к объединению** (`torch.cat`).

**Пошагово:**

1. **Цикл по каналам:**
   * `for i in range(input_tensor.shape[1])`:  Цикл проходит по каждому каналу входного тензора (`input_tensor`).

2. **Вычисление среднего по каналу:**
   * `torch.mean(input_tensor[:,i,:,:])`: Вычисляет среднее значение всех элементов в `i`-м канале (по всем элементам `batch`, "высоте" и "ширине").
   * Результат — скалярное значение (одно число), представляющее среднее значение по каналу.

3. **Добавление размерности:**
   * `unsqueeze(0)`:  Добавляет новую ось (размерность) на нулевую позицию, делая скалярное значение тензором с размерностью `(1,)`.

4. **Объединение тензоров:**
   * `torch.cat([...], dim=0)`: Объединяет полученные тензоры среднего значения для всех каналов вдоль первой оси (ось "batch").
   * Результат — тензор `mean` с размерностью `(input_channels,)`.

**Пример:**

Представьте, что у вас входной тензор `input_tensor` с размерностью `(2, 3, 4, 5)`.

* `i = 0`: Вычисляется среднее значение по первому каналу — это одно число.
* `unsqueeze(0)` делает это число тензором с размерностью `(1,)`.
* После прохождения по всем каналам (`i = 0, 1, 2`), `torch.cat` объединяет 3 тензора среднего значения вдоль первой оси, создавая тензор `mean` с размерностью `(3,)`.

**Почему `unsqueeze(0)` нужен?**

* **Совместимость с `torch.cat`:** `torch.cat` требует, чтобы тензоры, которые объединяются, имели одинаковое количество размерностей.  
* **Размерность для объединения:** `unsqueeze(0)` добавляет размерность "batch", чтобы тензор среднего значения был совместим с тензорами из других каналов, которые будут объединены в `torch.cat`.

**Вкратце:** `unsqueeze(0)` используется для создания тензоров среднего значения с правильной размерностью для последующего объединения в тензор `mean`.

# Объяснение для Способа 3

```python
normed_tensor = torch.stack([
    (input_tensor[:, i, :, :] - mean[i]) / torch.sqrt(var[i] + eps)
    for i in range(input_tensor.shape[1])], dim=1)
```

**Объяснение:**

1. **Список нормализованных тензоров:**
   * `[(input_tensor[:, i, :, :] - mean[i]) / torch.sqrt(var[i] + eps) for i in range(input_tensor.shape[1])]`:  Создается список, где каждый элемент — это тензор с нормализованными значениями для соответствующего канала.

2. **Объединение тензоров:**
   * `torch.stack([...], dim=1)`:  Создает новый тензор, "складывая" тензоры из списка вдоль второй оси (`dim=1`).

**Почему `torch.stack` лучше, чем `torch.cat` с `unsqueeze(1)`?**

* **Назначение `torch.stack`:**  `torch.stack` предназначен для объединения тензоров по новой оси, добавляя новую размерность. Это то, что вам нужно в этом случае.
* **`torch.cat` с `unsqueeze(1)`:**  `torch.cat` объединяет тензоры вдоль уже существующей оси. Вам нужно было добавлять новую ось с помощью `unsqueeze(1)`, чтобы `torch.cat` работал корректно.

**Вкратце:**  `torch.stack` — это более подходящая функция для объединения тензоров, когда требуется добавить новую размерность.

**Дополнительные сведения:**

* `torch.stack` создает новый тензор, объединяя тензоры по новой оси.
* `torch.cat` объединяет тензоры вдоль уже существующей оси.


In [ ]:
import torch
import torch.nn as nn

eps = 1e-3

input_channels = 3
batch_size = 3
height = 10
width = 10

batch_norm_2d = nn.BatchNorm2d(input_channels, affine=False, eps=eps)

input_tensor = torch.randn(batch_size, input_channels, height, width, dtype=torch.float)


def custom_batch_norm2d(input_tensor, eps):
    # Напишите в этом методе нормирование тензора.
    normed_tensor = (input_tensor - input_tensor.mean(axis=(0, 2, 3), keepdim=True)) / torch.sqrt(input_tensor.var(axis=(0, 2, 3), keepdim=True, unbiased=False) + eps)
    return normed_tensor


# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
norm_output = batch_norm_2d(input_tensor)
custom_output = custom_batch_norm2d(input_tensor, eps)
print(torch.allclose(norm_output, custom_output) and norm_output.shape == custom_output.shape)

True


In [ ]:
import torch

x = torch.randn(2, 2, 2, 2)

# Вычисление среднего значения по осям 0, 2 и 3, сохраняя размерности
y = x.mean(dim=(0, 2, 3), keepdim=True)
y = x.mean()

print(x.shape) # Вывод: torch.Size([2, 3, 4, 5])
print(x)
print(y.shape) # Вывод: torch.Size([1, 3, 1, 1])
print(y)

torch.Size([2, 2, 2, 2])
tensor([[[[-0.1896,  2.1385],
          [-0.4014,  0.2318]],

         [[-1.4736, -1.3380],
          [-2.0564,  0.6847]]],


        [[[-1.0613,  0.6353],
          [ 2.5046,  0.5319]],

         [[ 0.3663,  0.5198],
          [-0.8723,  1.7398]]]])
torch.Size([])
tensor(0.1225)


In [ ]:
#@title Как работает unsqueeze(0) и unsqueeze(1)
x = torch.randn(2, 2, 2, 2)

y0 = x.unsqueeze(0)
y1 = x.unsqueeze(3)

print(x.shape) # Вывод: torch.Size([2, 3, 4, 5])
print(x)
print(y0.shape) # Вывод: torch.Size([1, 2, 3, 4, 5])
print(y0)
print(y1.shape) # Вывод: torch.Size([2, 1, 3, 4, 5])
print(y1)
#

torch.Size([2, 2, 2, 2])
tensor([[[[ 0.2715,  1.6148],
          [ 1.1601, -0.3181]],

         [[ 1.2772, -1.0144],
          [ 0.5571, -0.4251]]],


        [[[ 1.4495,  1.4195],
          [-1.0405, -0.1200]],

         [[ 0.2292, -0.5486],
          [-0.0263, -0.6131]]]])
torch.Size([1, 2, 2, 2, 2])
tensor([[[[[ 0.2715,  1.6148],
           [ 1.1601, -0.3181]],

          [[ 1.2772, -1.0144],
           [ 0.5571, -0.4251]]],


         [[[ 1.4495,  1.4195],
           [-1.0405, -0.1200]],

          [[ 0.2292, -0.5486],
           [-0.0263, -0.6131]]]]])
torch.Size([2, 2, 2, 1, 2])
tensor([[[[[ 0.2715,  1.6148]],

          [[ 1.1601, -0.3181]]],


         [[[ 1.2772, -1.0144]],

          [[ 0.5571, -0.4251]]]],



        [[[[ 1.4495,  1.4195]],

          [[-1.0405, -0.1200]]],


         [[[ 0.2292, -0.5486]],

          [[-0.0263, -0.6131]]]]])
